姓名:簡晟棋

學號:110062209

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [14]:
!gdown --id 1CueEO9KKA5QGFB2uijp_mPsw9_jB3uFU
!unzip -qq -u dataset.zip

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1CueEO9KKA5QGFB2uijp_mPsw9_jB3uFU
From (redirected): https://drive.google.com/uc?id=1CueEO9KKA5QGFB2uijp_mPsw9_jB3uFU&confirm=t&uuid=045b3621-9534-4896-8bce-6b93d24c2424
To: /content/dataset.zip
100% 97.2M/97.2M [00:02<00:00, 42.1MB/s]


我曾經試過對content內容用countvectorizer與tfidfvectorizer,但結果都不盡理想。因此，我認為比起內容，能見度或許更重要，於是只提取了topic、發布時間(詳細到小時)這兩種features:

發布時間:影響新聞能見度

topic:比content更簡潔，比channel更詳細。除了能夠當作內容key word外，還能代替channel影響新聞能見度

In [15]:
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download("wordnet")
stop = stopwords.words('english')

day_to_num = {'mon':1,'tue':2,'wed':3,'thu':4,'fri':5,'sat':6,'sun':7}
month_to_num = {'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11,'dec':12}

def preprocessor(text):
    soup = BeautifulSoup(text, 'html.parser')

    a_list = soup.find('footer').find_all('a')
    topic_list = [a.get_text().strip().lower() for a in a_list]
    topic = ' '.join([re.sub('\s+', '_', t) for t in topic_list])

    time_tag = soup.find('time')
    if time_tag and 'datetime' in time_tag.attrs:
        date_time = time_tag['datetime']
    else:
        date_time = 'Thu, 10 Oct 2024 00:00:00'
    time_obj = re.search('([\w]+),\s+([\d]+)\s+([\w]+)\s+([\d]+)\s+([\d]+)', date_time)
    day, date, month, year, hour = time_obj.groups()
    day, month = day_to_num[day.lower()], month_to_num[month.lower()]

    return topic, day, date, month, year, hour

def tokenizer_lem_nostop(text):
    wnl = WordNetLemmatizer()
    return [wnl.lemmatize(w) for w in re.split('\s+', text.strip()) if w not in stop and re.match('[a-zA-Z]+', w)]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
import pandas as pd
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')
feature_train = []
feature_test = []
for text in train['Page content']:
    feature_train.append(preprocessor(text))
for text in test['Page content']:
    feature_test.append(preprocessor(text))

In [17]:
columns=['Topic', 'Day', 'Date', 'Month', 'Year', 'Hour']
df_train = pd.DataFrame(
    feature_train,
    columns=columns
)
df_test = pd.DataFrame(
    feature_test,
    columns=columns
)

對於topic,我嘗試tfidfvectorizer與countvectorizer兩種方式轉換

In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer

trans_count = ColumnTransformer([('Topic', CountVectorizer(tokenizer=tokenizer_lem_nostop),'Topic')],remainder='passthrough')
trans_tfidf = ColumnTransformer([('Topic', TfidfVectorizer(tokenizer=tokenizer_lem_nostop),'Topic')],remainder='passthrough')

In [19]:
from sklearn.model_selection import train_test_split

y_train_raw = (train['Popularity'].values==1).astype(int)

X_train, X_valid, y_train, y_valid = train_test_split(df_train,y_train_raw,test_size=0.2,random_state=0)
X_train = pd.DataFrame(X_train,columns=columns)
X_valid = pd.DataFrame(X_valid,columns=columns)

而在model的部分,我試了LightBoost,XGBoost,RandomForest等classifier。

In [20]:
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

In [ ]:
lgbm_tf = Pipeline([('vect', trans_tfidf),('clf', LGBMClassifier(learning_rate=0.01,n_estimators=300,verbose=0))])

scores = cross_val_score(estimator=lgbm_tf, X=df_train, y=y_train_raw, cv=5, scoring='roc_auc')
print('score: %.3f (+/-%.3f)' % (scores.mean(), scores.std()))

lgbm_tf.fit(X_train, y_train)
train_score = lgbm_tf.predict_proba(X_train)[:,1]
print(f'train score:{roc_auc_score(y_train,train_score)}')
valid_score = lgbm_tf.predict_proba(X_valid)[:,1]
print(f'valid score:{roc_auc_score(y_valid,valid_score)}')

score: 0.600 (+/-0.009)
train score:0.6858100659944703
valid score:0.591644449980987


In [21]:
lgbm_ct = Pipeline([('vect', trans_count),('clf', LGBMClassifier(learning_rate=0.01,n_estimators=300,verbose=0))])

scores = cross_val_score(estimator=lgbm_ct, X=df_train, y=y_train_raw, cv=5, scoring='roc_auc')
print('score: %.3f (+/-%.3f)' % (scores.mean(), scores.std()))

lgbm_ct.fit(X_train, y_train)
train_score = lgbm_ct.predict_proba(X_train)[:,1]
print(f'train score:{roc_auc_score(y_train,train_score)}')
valid_score = lgbm_ct.predict_proba(X_valid)[:,1]
print(f'valid score:{roc_auc_score(y_valid,valid_score)}')

score: 0.604 (+/-0.007)
train score:0.6697293570976751
valid score:0.5962876435099319


In [ ]:
xgb_tf = Pipeline([('vect', trans_tfidf),('clf', XGBClassifier(eta=0.01,n_estimators=300,verbose=0))])

scores = cross_val_score(estimator=xgb_tf, X=df_train, y=y_train_raw, cv=5, scoring='roc_auc')
print('score: %.3f (+/-%.3f)' % (scores.mean(), scores.std()))

xgb_tf.fit(X_train, y_train)
train_score = xgb_tf.predict_proba(X_train)[:,1]
print(f'train score:{roc_auc_score(y_train,train_score)}')
valid_score = xgb_tf.predict_proba(X_valid)[:,1]
print(f'valid score:{roc_auc_score(y_valid,valid_score)}')

score: 0.595 (+/-0.009)
train score:0.682518053502863
valid score:0.5867981723477438


In [ ]:
xgb_ct = Pipeline([('vect', trans_count),('clf', XGBClassifier(eta=0.01,n_estimators=300,verbose=0))])

scores = cross_val_score(estimator=xgb_ct, X=df_train, y=y_train_raw, cv=5, scoring='roc_auc')
print('score: %.3f (+/-%.3f)' % (scores.mean(), scores.std()))

xgb_ct.fit(X_train, y_train)
train_score = xgb_ct.predict_proba(X_train)[:,1]
print(f'train score:{roc_auc_score(y_train,train_score)}')
valid_score = xgb_ct.predict_proba(X_valid)[:,1]
print(f'valid score:{roc_auc_score(y_valid,valid_score)}')

score: 0.599 (+/-0.007)
train score:0.6672210010720567
valid score:0.5873803380565218


In [ ]:
rf_tf = Pipeline([('vect', trans_tfidf),('clf', RandomForestClassifier(n_estimators=300,verbose=0))])

scores = cross_val_score(estimator=rf_tf, X=df_train, y=y_train_raw, cv=5, scoring='roc_auc')
print('score: %.3f (+/-%.3f)' % (scores.mean(), scores.std()))
rf_tf.fit(X_train, y_train)
train_score = rf_tf.predict_proba(X_train)[:,1]
print(f'train score:{roc_auc_score(y_train,train_score)}')
valid_score = rf_tf.predict_proba(X_valid)[:,1]
print(f'valid score:{roc_auc_score(y_valid,valid_score)}')

score: 0.579 (+/-0.010)
train score:0.9999602872230277
valid score:0.5865560285751897


In [ ]:
rf_ct = Pipeline([('vect', trans_count),('clf', RandomForestClassifier(n_estimators=300,verbose=0))])

scores = cross_val_score(estimator=rf_ct, X=df_train, y=y_train_raw, cv=5, scoring='roc_auc')
print('score: %.3f (+/-%.3f)' % (scores.mean(), scores.std()))
rf_ct.fit(X_train, y_train)
train_score = rf_ct.predict_proba(X_train)[:,1]
print(f'train score:{roc_auc_score(y_train,train_score)}')
valid_score = rf_ct.predict_proba(X_valid)[:,1]
print(f'valid score:{roc_auc_score(y_valid,valid_score)}')

score: 0.578 (+/-0.011)
train score:0.9999567855607812
valid score:0.5856666850733445


最終，我選擇用countvectorizer轉換topic，並用LightBoost當主要的classifier，因為該組合cross validation平均分數最高

In [22]:
import os
if not os.path.exists("output/") : os.mkdir("output/")
test_id = test['Id']
Y_test = lgbm_ct.predict_proba(df_test)[:,1]
data = {
    'Id': test_id,
    'Popularity': Y_test
}
pd.DataFrame(data).to_csv('output/result.csv', index=False)

從上面實驗可以得知比起內容是什麼,topic與發佈時間這些影響能見度的因素才是決定新聞popularity的關鍵。
